In [1]:
import pandas as pd
import json
import mapply
import re
pd.set_option('display.max_colwidth', None)  # Set maximum width to terminal width
pd.set_option('display.expand_frame_repr', True)  # Expand DataFrame representation
mapply.init(
    n_workers=-1,
    chunk_size=1000,
    max_chunks_per_worker=0,
    progressbar=True
)

/home/user/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [22]:
# Set the path to your  JSON file
json_file_path = "/home/user/ItemDataQuality/data/meta_All_Beauty.json"

#load file
df = pd.read_json(json_file_path,lines=True)

# # or load in chunks if needed
# # Define the chunk size (adjust as needed)
# chunk_size = 10000

# # Initialize an empty list and Append the  chunk to the list
# chunk_dataframes = [chunk for chunk in pd.read_json(json_file_path, lines=True, chunksize=chunk_size) ]

# # Concatenate all cleaned chunks into a single DataFrame
# df = pd.concat(chunk_dataframes, ignore_index=True)

In [23]:
df.shape

(32892, 19)

In [25]:
df = df.drop(['category','tech1','fit','tech2','similar_item','date','also_view','also_buy'],axis=1)

In [26]:
df = df.applymap(lambda x: '; '.join(map(str, x)) if x is not None and isinstance(x, list) else x)

/tmp/ipykernel_6724/2736066085.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: '; '.join(map(str, x)) if x is not None and isinstance(x, list) else x)


In [27]:
df['rank'] = df['rank'].mapply(lambda x: re.sub('[^a-zA-Z0-9&, ]|amp', '', x) if x is not None and isinstance(x, str) else x)

/home/user/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'Series.swapaxes' is deprecated and will be removed in a future version. Please use 'Series.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 32/32 [00:00<00:00, 57.56it/s]


In [28]:
normalized_data = pd.json_normalize(df['details'])
normalized_data.head()

,ASIN:,Shipping Weight:,\n Item Weight: \n,Item model number:,UPC:,\n Product Dimensions: \n,Discontinued by manufacturer:,Domestic Shipping:,International Shipping:,Batteries,Shipping Advisory:,ASIN:
0,6546546450,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,7178680776,0.3 ounces (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7250468162,3.5 ounces (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,7367905066,1.8 ounces,1.76 ounces,0000011837,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,7414204790,12 ounces (,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
normalized_data.columns

Index(['ASIN: ', 'Shipping Weight:', '\n    Item Weight: \n    ',
       'Item model number:', 'UPC:', '\n    Product Dimensions: \n    ',
       'Discontinued by manufacturer:', 'Domestic Shipping: ',
       'International Shipping: ', 'Batteries', 'Shipping Advisory:', 'ASIN:'],
      dtype='object')

In [11]:
#Assign None if below three columns not present in normalized data
# normalized_data['product_dimensions']=None
# normalized_data['item_weight']=None
# normalized_data['shipping_weight']=None

In [30]:
# Add the normalized data back to the original DataFrame based on the index
df = pd.concat([df, normalized_data], axis=1, join='inner')

In [31]:
df['category']='All_Beauty'

In [32]:
# Clean and standardize DataFrame column names
df.columns = (df.columns
                           .str.strip()  # Remove leading/trailing spaces
                           .str.lower()  # Convert to lowercase (optional for consistency)
                           .str.replace(':', '') # Remove colons characters
                           .str.replace('\n', '') # Remove newline characters
                           .str.replace('.', '_') # Replace periods with underscores
                           .str.replace(' ', '_'))  # Replace spaces with underscores

In [33]:
#drop duplicate columns
df = df.loc[:, ~df.columns.duplicated()]

In [34]:
df.columns

Index(['description', 'title', 'brand', 'feature', 'rank', 'details',
       'main_cat', 'price', 'asin', 'imageurl', 'imageurlhighres',
       'shipping_weight', 'item_weight', 'item_model_number', 'upc',
       'product_dimensions', 'discontinued_by_manufacturer',
       'domestic_shipping', 'international_shipping', 'batteries',
       'shipping_advisory', 'category'],
      dtype='object')

In [ ]:
df = df[['asin','category','main_cat','title','description', 
        'brand','price','product_dimensions','item_weight', 'shipping_weight',
        'imageurl', 'imageurlhighres','feature','rank']]

In [1]:
df.head(3)

NameError: name 'df' is not defined

##### Define a file name and save it to the Parquet file in the GCS bucket.

In [21]:

df.to_parquet('gs://data_quality_final/parquet_data_files/meta_All_Beauty.parquet',index=False)